In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

from torchvision.datasets import MNIST

In [2]:
mnist_n_objects = 4000

In [3]:
mnist_test_dataset = MNIST('./mnist', train=False, download=True)
mnist_train_dataset = MNIST('./mnist', train=True, download=False)

mnist_labels_test = np.array(mnist_test_dataset.targets)
mnist_labels_train = np.array(mnist_train_dataset.targets)

mnist_images_test = mnist_test_dataset.data.numpy()
mnist_images_train = mnist_train_dataset.data.numpy()

mnist_images_test, _, mnist_labels_test, _ = train_test_split(
    mnist_images_test, mnist_labels_test,
    train_size=mnist_n_objects, stratify=mnist_labels_test, random_state=6886
)
mnist_images_train, _, mnist_labels_train, _ = train_test_split(
    mnist_images_train, mnist_labels_train,
    train_size=mnist_n_objects, stratify=mnist_labels_train, random_state=6886
)

mnist_data_test = (mnist_images_test.astype(np.float32) / 255.0).reshape([mnist_images_test.shape[0], -1])
mnist_data_train = (mnist_images_train.astype(np.float32) / 255.0).reshape([mnist_images_train.shape[0], -1])


100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:08<00:00, 1183157.71it/s]


Extracting ./mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist\MNIST\raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 3313792.75it/s]


Extracting ./mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist\MNIST\raw



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1480965.86it/s]


Extracting ./mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist\MNIST\raw



In [4]:
fraction_of_labeled = 0.01

In [5]:
mnist_data_labeled_train, mnist_data_unlabeled_train, mnist_labels_labeled_train, mnist_labels_unlabeled_train = train_test_split(
    mnist_data_train, mnist_labels_train,
    train_size=fraction_of_labeled, stratify=mnist_labels_train, random_state=6886
)

In [6]:
# Замена на -1
mnist_labels_unlabeled_train = np.full_like(mnist_labels_unlabeled_train, -1)

In [7]:
# Разбиение тестовой выборки на тестовую и валидационную части
mnist_data_test, mnist_data_val, mnist_labels_test, mnist_labels_val = train_test_split(
    mnist_data_test, mnist_labels_test,
    test_size=0.7, stratify=mnist_labels_test, random_state=6886
)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [9]:
logistic_regression = LogisticRegression(max_iter=10000)
logistic_regression.fit(mnist_data_labeled_train, mnist_labels_labeled_train)


LogisticRegression(max_iter=10000)

In [10]:
predictions = logistic_regression.predict(mnist_data_test)

In [11]:
accuracy = accuracy_score(mnist_labels_test, predictions)
print("Accuracy on the test set:", accuracy)

Accuracy on the test set: 0.64


In [12]:
from sklearn.cluster import KMeans
from collections import Counter

In [13]:
n_clusters = 10

In [14]:
kmeans = KMeans(n_clusters=n_clusters, random_state=6886)
kmeans.fit(mnist_data_train)
cluster_labels = kmeans.predict(mnist_data_labeled_train)

In [15]:
# Присвоение меток кластерам
cluster_to_label = {}
for cluster in range(n_clusters):
    cluster_indices = np.where(cluster_labels == cluster)[0]
    #cluster_labels_in_cluster = mnist_labels_train[cluster_indices]
    cluster_labels_in_cluster = mnist_labels_labeled_train[cluster_indices]
    most_common_label = Counter(cluster_labels_in_cluster).most_common(1)[0][0]
    cluster_to_label[cluster] = most_common_label

cluster_assigned_labels = np.array([cluster_to_label[cluster] for cluster in cluster_labels])

IndexError: list index out of range

In [16]:
cluster_labels

array([2, 5, 8, 6, 6, 4, 6, 2, 9, 2, 9, 5, 7, 5, 9, 5, 1, 0, 7, 1, 5, 1,
       6, 7, 0, 1, 4, 0, 5, 6, 0, 9, 0, 1, 1, 5, 8, 0, 7, 6])

In [17]:
unlabled_predict = kmeans.predict(mnist_data_train)

In [18]:
unlabled_labels = []
for cluster in unlabled_predict:
  unlabled_labels.append(cluster_to_label[cluster])

KeyError: 3

In [19]:
logistic_regression_cluster = LogisticRegression(max_iter=1000)
logistic_regression_cluster.fit(mnist_data_train, unlabled_labels)

ValueError: Found input variables with inconsistent numbers of samples: [4000, 0]

In [ ]:
predictions_cluster = logistic_regression_cluster.predict(mnist_data_test)

In [ ]:
accuracy_cluster = accuracy_score(mnist_labels_test, predictions_cluster)
print("Accuracy on the test set with clustering:", accuracy_cluster)
